In [1]:
from libs.chatglm3 import ChatGLM3

In [2]:
MODEL_PATH = 'D:\Github\Repositories\models\huggingface\chatglm3-6b'
llm = ChatGLM3()
llm.load_model(MODEL_PATH)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
from langchain_community.document_loaders import DirectoryLoader

In [4]:
loader = DirectoryLoader('./static/books')
# documents = loader.load()   

In [5]:
import pickle

In [6]:
# with open('data.pickle', 'wb') as f:
#     pickle.dump(documents, f)

In [7]:
with open('data.pickle', 'rb') as f:
    documents = pickle.load(f)

In [8]:
from langchain.text_splitter import CharacterTextSplitter

In [9]:
text_splitter = CharacterTextSplitter(chunk_size=256, chunk_overlap=8)

In [10]:
splitted_documents = text_splitter.split_documents(documents)

Created a chunk of size 432, which is longer than the specified 256
Created a chunk of size 1388, which is longer than the specified 256
Created a chunk of size 513, which is longer than the specified 256
Created a chunk of size 347, which is longer than the specified 256
Created a chunk of size 422, which is longer than the specified 256
Created a chunk of size 290, which is longer than the specified 256
Created a chunk of size 269, which is longer than the specified 256
Created a chunk of size 349, which is longer than the specified 256
Created a chunk of size 488, which is longer than the specified 256
Created a chunk of size 274, which is longer than the specified 256
Created a chunk of size 410, which is longer than the specified 256
Created a chunk of size 313, which is longer than the specified 256
Created a chunk of size 259, which is longer than the specified 256
Created a chunk of size 273, which is longer than the specified 256
Created a chunk of size 382, which is longer th

In [11]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [12]:
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(model_name='shibing624/text2vec-base-chinese',encode_kwargs=encode_kwargs)

In [13]:
from langchain.vectorstores import Chroma

In [14]:
db = Chroma.from_documents(splitted_documents, embeddings)
# db.persist()

In [15]:
from langchain.chains import RetrievalQA

In [16]:
retriever = db.as_retriever()

In [18]:
# custom_rag_prompt = PromptTemplate.from_template(template=template)


In [19]:
# def format_docs(docs):
#     return "\n\n".join(document.page_content for document in documents)

In [19]:
# from langchain.chains import ConversationalRetrievalChain

In [21]:
# qa = ConversationalRetrievalChain.from_llm(
#         llm=llm, retriever=retriever,
#         return_source_documents=True,
#         combine_docs_chain_kwargs={'prompt': custom_rag_prompt})

In [17]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever
)

In [18]:
qa.invoke(' 医学难以解释的症状是什么？')

d:\GitHub\envs\breastvsp\lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
d:\GitHub\envs\breastvsp\lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
d:\GitHub\envs\breastvsp\lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': ' 医学难以解释的症状是什么？',
 'result': '医学难以解释的症状可能包括焦虑、抑郁、失眠、头晕、头痛、吸困难和腹痛、关注过度、强化躯体症状、新的症状、精神心理问题、不良反应、应激、无法放松、变得容易烦躁或急躁、感到似乎将有可怕的事情发生而害怕等。这些症状可能与肿瘤、心理健康问题、治疗过程相关，也可能与患者的日常功能损害、治疗满意度、合作治疗方式、医患沟通、治疗获得信息等方面有关。'}

In [19]:
qa.invoke('骨痛是什么？')

d:\GitHub\envs\breastvsp\lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
d:\GitHub\envs\breastvsp\lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
d:\GitHub\envs\breastvsp\lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': '骨痛是什么？',
 'result': '骨痛是一种骨骼疾病，患者通常会感到骨骼部位的疼痛。骨痛可能是由各种原因引起的，如骨折、骨关节炎、骨质疏松等。在处理骨痛时，专家建议可以通过检测进行诊断，如影像学、B超和骨髓检测，以确定患者是否存在骨转移。治疗骨痛的方法可能因患者具体情况而异，可能包括停止使用相关药物、进行氧疗、运动疗法等。同时，专家建议对患者进行体重管理，以肥胖患者为例，需要进行健康饮食和运动干预。在治疗骨痛的过程中，应根据患者具体情况进行针对性的处理。'}

In [20]:
import torch

In [23]:
torch.cuda.empty_cache()

In [22]:
qa.invoke('骨痛怎么办？')

d:\GitHub\envs\breastvsp\lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
d:\GitHub\envs\breastvsp\lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
d:\GitHub\envs\breastvsp\lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': '骨痛怎么办？',
 'result': '骨痛的处理可以参考以下专家建议：\n\n1. 诊断：通过影像学、B超和骨髓检测进行诊断，以确定患者是否存在骨转移。\n2. 治疗：停止使用相关药物后，相关骨痛症状可能消失。对于未出现肌肉骨骼疼痛的患者，加强患者对可能出现的肌肉骨骼疼痛的了解，鼓励参加运动和力量训练、瑜伽等。肥胖患者需要进行体重管理。\n3. 运动干预：针对出现肌肉骨骼疼痛的患者，可以通过针灸（包括有氧训练、力量训练、瑜伽等）来改善症状。AI可以缓解患者的肌肉骨骼疼痛。'}